In [1]:
import sys
from openmm import openmm as mm, app
from mstk.topology import Molecule, Topology
from mstk.forcefield import ForceField, ZftTyper
from mstk.simsys import System
from mstk.wrapper import Packmol
from mstk import ommhelper as oh

In [2]:
# Create topology from SMILES
benzene = Molecule.from_smiles('c1ccccc1')
water = Molecule.from_smiles('O')
top = Topology([benzene, water])

# Assign atom types as defined in `data/forcefield/primitive.zft`
typer = ZftTyper('primitive.zft')
typer.type(top)

# Build a bulk liquid simulation box with Packmol
packmol = Packmol('packmol')
top.cell.set_box([4.0, 4.0, 4.0])
top.scale_with_packmol([100, 1000], packmol=packmol)

# Assign force field parameters as defined in `data/forcefield/primitive.zff`
ff = ForceField.open('primitive.zff')
ff.assign_charge(top)
system = System(top, ff)

# Generate input files for LAMMPS, GROMACS and NAMD
# system.export_lammps()
# system.export_gromacs()
# system.export_namd()

# Generate OpenMM system and topology
omm_sys = system.to_omm_system()
omm_top = top.to_omm_topology()

In [7]:
# Init OpenMM simulation
integrator = mm.LangevinMiddleIntegrator(300, 1.0, 0.002)
platform = mm.Platform.getPlatformByName('CUDA')
properties = {'Precision': 'Mixed'}
sim = app.Simulation(omm_top, omm_sys, integrator, platform, properties)
sim.reporters.append(oh.StateDataReporter(sys.stdout, 1000, forceGroups=[0,1,2,3,4,5,6,7]))
sim.context.setPositions(top.positions)

In [8]:
sim.step(10000)

#"Step"	"Temp"	"E_pot"	"Lx"	"Ly"	"Lz"	"Density"	"Speed"	"E_0_CMMotionRemover"	"E_1_Bond"	"E_2_Angle"	"E_3_Dihedral"	"E_4_Improper"	"E_5_vdW"	"E_6_Coulomb"	"E_7_"
1000	422.28	-28131.0325	4.0000	4.0000	4.0000	0.6701	0	0.0	925.1385958477446	1050.5873788215304	962.823075601675	95.32552105978436	6361.746989757108	-37526.653857005615	0.0
2000	339.02	-34997.5426	4.0000	4.0000	4.0000	0.6701	330	0.0	907.2138834473368	1044.2075855464916	980.3761578219375	98.22440142132075	8461.045163182958	-46488.60948139979	0.0
3000	314.85	-37546.7143	4.0000	4.0000	4.0000	0.6701	339	0.0	1025.5265962855055	1047.557320074509	1078.0265859313286	104.8619283705201	9108.92813014646	-49911.61495575178	0.0
4000	303.74	-38644.1277	4.0000	4.0000	4.0000	0.6701	339	0.0	995.3156282119526	1055.2746038790565	1009.8599338456434	105.84240799430461	9221.104499448646	-51031.52474673436	0.0
5000	301.98	-39054.1789	4.0000	4.0000	4.0000	0.6701	337	0.0	993.689534151029	1051.2243913859422	1058.57555849264	110.67350387180664	9007.79650